### 이전 코드에서 생성한 데이터 불러오기

In [1]:
import numpy as np
# continual learning을 위한 데이터셋

train_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
train_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_A.npy")
train_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_B.npy")
train_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_train_data_C.npy")

train_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_A.npy")
train_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_B.npy")
train_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_train_data_C.npy")

val_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
val_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_A.npy")
val_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_B.npy")
val_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_val_data_C.npy")

val_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_A.npy")
val_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_B.npy")
val_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_val_data_C.npy")

test_data_level3 = {"A":[0,0],"B":[0,0],"C":[0,0]}
test_data_level3["A"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_A.npy")
test_data_level3["B"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_B.npy")
test_data_level3["C"][0] = np.load("../dataset/continual_learning_dataset/X_continual_test_data_C.npy")

test_data_level3["A"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_A.npy")
test_data_level3["B"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_B.npy")
test_data_level3["C"][1] = np.load("../dataset/continual_learning_dataset/y_continual_test_data_C.npy")


# joint training 을 위한 데이터셋

X_train_3 = np.load("../dataset/joint_training_dataset/X_joint_train_data.npy")
y_train_3 = np.load("../dataset/joint_training_dataset/y_joint_train_data.npy")
X_val_3 = np.load("../dataset/joint_training_dataset/X_joint_val_data.npy")
y_val_3 = np.load("../dataset/joint_training_dataset/y_joint_val_data.npy")
X_test_3 = np.load("../dataset/joint_training_dataset/X_joint_test_data.npy")
y_test_3 = np.load("../dataset/joint_training_dataset/y_joint_test_data.npy")


### 데이터로더 래핑

In [2]:
# 상위 디렉토리로 작업환경 이동
import os
os.chdir(os.path.join(globals()['_dh'][0], ".."))

In [3]:
from dfb.dataset import *
from dfb.processing import *
import experiment
import torchvision.transforms as transforms

model_name = "wdcnn2"

sample_length = experiment.get_sample_length(model_name)
tf_data = experiment.get_transform(model_name)
tf_data = transforms.Compose(tf_data)
tf_label = NpToTensor()
batch_size = 128
num_worker = 4

data_handler = DatasetHandler()

data_handler.assign(
    X_train_3, y_train_3, X_val_3, y_val_3, X_test_3, y_test_3,
    sample_length, "3", tf_data, tf_label, batch_size, num_worker
)

/data/home/happy113200/anaconda3/envs/cfb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 모델 가져오기

In [4]:

# 모델 수정되는 경우 해당 셀에서 바로 갱신되도록 autoreload사용
%load_ext autoreload
%autoreload 2

from dfb.model.wdcnn2 import *
# 클래스 수는 맘대로 지정

# 클래스 수 10개짜리 모델 만들기
model = WDCNN2(n_classes=3)

### 훈련하기

In [5]:
from dfb.trainmodule import *

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

# ticnn 플래그는 일단 신경쓸 필요 X
training_module = PlModule(model, optimizer, loss, True)

n_steps_d = len(data_handler.dataloaders["3"]["train"].dataset) // (batch_size)

# val_loss 가 가장 작은 모델을 저장하는 콜백
callback = pl.callbacks.ModelCheckpoint(monitor="val_loss",
                                        dirpath=f"./logs/test/best_model",
                                        filename=f"model",
                                        save_top_k=1,
                                        mode="min")
trainer = pl.Trainer(
    gpus = [0],
    max_epochs=100,
    val_check_interval= n_steps_d,
    default_root_dir="./logs/test",
    callbacks=[callback]
)

result = trainer.fit(model=training_module,
                     train_dataloaders=data_handler.dataloaders["3"]["train"],
                     val_dataloaders=data_handler.dataloaders["3"]["val"])


/data/home/happy113200/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/data/home/happy113200/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /data/home/happy113200/DeepFault/logs/test/best_model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | WDCNN2           | 32.0 K
1 | loss_fn | CrossEntropyLoss |

/data/home/happy113200/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  96%|█████████▌| 23/24 [00:00<00:00, 32.11it/s, loss=1.03, v_num=55]

2023-07-17 03:54:28.389224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 03:54:28.562321: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-17 03:54:29.250562: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.2/lib64
2023-07-17 03:54:29.250651: W tensorflow/compiler/xla/strea

Epoch 95:   8%|▊         | 2/24 [00:00<00:02,  7.96it/s, loss=0.02, v_num=55, val_loss=0.0453, val_acc=0.985, train_loss=0.0212, train_acc=0.994]   

### 테스트하기

In [ ]:
training_module.load_from_checkpoint(f"./logs/test/best_model/model-v8.ckpt",
                                            model=model, optimizer=optimizer,
                                            loss_fn=loss)
result = trainer.test(model=training_module, dataloaders=data_handler.dataloaders["3"]["test"])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5]


Testing DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 99.83it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.852393627166748
        test_loss            0.326139360666275
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
